In [2]:
%pip install datasets pandas spacy tqdm

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached spacy-3.8.7-cp311-cp311-macosx_11_0_arm64.whl.metadata (27 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.13-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.2 kB)
  Using cached cymem-2.0.11-cp311-cp311-macosx_11_0_arm64.whl.metadata (8.5 kB)
  Using cached preshed-3.0.10-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.4 kB)
  Using cached thinc-8.3.6-cp311-cp311-macosx_11_0_arm64.whl.metadat

In [3]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 71.8 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [4]:
from datasets import load_dataset

ds = load_dataset("trl-lib/tldr", split="train")

/Users/sophiavo/ML Tasks/Reddit TLDR/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 6553/6553 [00:00<00:00, 678146.41 examples/s]


In [5]:
import re

def split_prompt(example):
    text = example["prompt"]
    # Regex with DOTALL so that POST: can span multiple lines
    m = re.match(
        r"SUBREDDIT:\s*(?P<subreddit>.+?)\s+TITLE:\s*(?P<title>.+?)\s+POST:\s*(?P<post>.+?)\s+TL;DR:",
        text,
        flags=re.DOTALL,
    )
    if not m:
        return {"subreddit": None, "title": None, "post": text}
    return m.groupdict()

ds = ds.map(split_prompt, remove_columns=["prompt"])

Map: 100%|██████████| 116722/116722 [00:04<00:00, 24770.74 examples/s]


In [6]:
# ‘completion’ → ‘tldr’, drop any rows missing required fields
ds = ds.rename_column("completion", "tldr")
ds = ds.filter(lambda x: x["subreddit"] and x["title"] and x["post"] and x["tldr"])

Filter: 100%|██████████| 116722/116722 [00:00<00:00, 318727.21 examples/s]


In [7]:
import unicodedata

def clean_text(example):
    for col in ["title", "post", "tldr"]:
        text = example[col].strip()
        text = unicodedata.normalize("NFKC", text)
        example[col] = text
    return example

ds = ds.map(clean_text)

Map: 100%|██████████| 116722/116722 [00:04<00:00, 26842.21 examples/s]


In [8]:
import spacy
from tqdm.auto import tqdm

nlp = spacy.load("en_core_web_md")

def compute_similar(example):
    doc_post = nlp(example["post"])
    doc_tldr = nlp(example["tldr"])
    tldr_text = example["tldr"].lower()
    similar = {}
    for chunk in doc_post.noun_chunks:
        phrase = chunk.text.strip().lower()
        if len(phrase) < 3:
            continue
        # Binary importance if phrase appears in the TL;DR summary
        important = 1 if phrase in tldr_text else 0
        # Similarity score via spaCy vectors
        sim_score = float(doc_tldr.similarity(nlp(phrase)))
        if sim_score >= 0.75: # Threshold for similarity
            important = 1
        similar[phrase] = (important, sim_score)
    return {"similar": similar}

# Apply with a progress bar
records = []
for row in tqdm(ds, total=len(ds)):
    rec = dict(row)
    rec.update(compute_similar(row))
    records.append(rec)

# Convert back into a Dataset
from datasets import Dataset
ds = Dataset.from_pandas(pd.DataFrame(records))


  0%|          | 52/116722 [00:08<5:06:10,  6.35it/s]/var/folders/zj/wg5c8ngn6j32wt_qg9x5p9bc0000gn/T/ipykernel_73944/4176085561.py:18: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim_score = float(doc_tldr.similarity(nlp(phrase)))
  7%|▋         | 8028/116722 [15:46<3:33:36,  8.48it/s]


KeyboardInterrupt: 

In [ ]:
def keep_top_k(example, k: int = 30):
    items = sorted(
        example["similar"].items(),
        key=lambda kv: kv[1][1],
        reverse=True
    )[:k]
    example["similar"] = dict(items)
    return example

ds = ds.map(keep_top_k)

ds = ds.filter(lambda x: len(x["similar"]) > 0)

In [ ]:
import json

# Convert the Dataset to a pandas DataFrame
df = ds.to_pandas()

# Serialize the nested `similar` dict into JSON strings
df['similar'] = df['similar'].apply(json.dumps)

# Write out to CSV (no index column)
df.to_csv("tldr_preprocessed.csv", index=False)

# loading the preprocessed data later:
# import pandas as pd
# df = pd.read_csv("tldr_preprocessed.csv")
# df['similar'] = df['similar'].apply(json.loads)